- [ ] Write down all the libraries that we need to install
- [ ] Add Try catch for exceptions when moving to .py file

## Important Notes
### Packages to install :
- pip install sounddevice
- pip install numpy
- Install ffmpeg (needed for intsalling local version of whisper)
    - On Ubuntu or Debian
      sudo apt update && sudo apt install ffmpeg
    - On Arch Linux
      sudo pacman -S ffmpeg
    - On MacOs
      brew install ffmpeg
    - On Windows
      winget install ffmpeg
- pip install -U openai-whisper
- pip install anthropic
- pip install dotenv
- pip install python-dotenv
- pip install openai
- pip install pyaudio
- pip install keyboard
- Installing pytorch
  - Install Cuda Toolkit (https://docs.nvidia.com/cuda/cuda-installation-guide-microsoft-windows/index.html) -> Only needed for GPU acceleration
  - Install Cudnn (https://developer.nvidia.com/cudnn) -> Only needed for GPU acceleration
  - pInstall pytorch with or without cuda (https://pytorch.org/get-started/locally/)
- pip install PyMuPDF -> To read the pdf files like CV's
### Instructions :
- There needs to be a .env in this notebook's working directory, which contains the api keys.

In [1]:
# Imports
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
import whisper
from joblib import load, dump
from src.utils.anthropicwrapper import ClaudeChatCV
from dotenv import load_dotenv, find_dotenv
import os
from openai import OpenAI
import pyaudio
import threading
import tkinter as tk
from threading import Thread, Event
from tkinter import messagebox
import time

load_dotenv(os.path.join(os.path.dirname(os.getcwd()), ".env"))

ModuleNotFoundError: No module named 'AnthropicWrapper'

In [ ]:
# --- Settings ---
FS = 44100               # Sampling frequency
THRESHOLD = 50          # Volume threshold for silence (adjust this)
SILENCE_DURATION = 1.5   # Seconds of silence before stopping (adjust this)
CHUNK_SIZE = 1024        # Process audio in chunks for efficiency
SPEAKING_SPEED = 1.1     # Speed of speaking
PASS_PERCENTAGE = 50     # The liniency of the interview

In [ ]:
# --- Globals ---
pause_loop = False
end_loop = False
unixtime = str(time.time())[:10]
print(unixtime)
human_audio_n = 0

1723753223


In [ ]:
# Global variables (consider using a class to manage them better)
pause_loop = False
end_loop = False
FS = 44100
CHUNK_SIZE = 1024
SPEAKING_SPEED = 1.0
PASS_PERCENTAGE = 0.7

# Global variables for job details
job_role = None
candidate_skill = None
role_description = None


In [ ]:
# Global event to signal when settings are updated
settings_updated = Event()

# --- Function to create the first UI (Job Details and Settings) ---
def create_ui_1():
    global FS, CHUNK_SIZE, SPEAKING_SPEED, PASS_PERCENTAGE
    global job_role, candidate_skill, role_description 

    def update_settings():
        global FS, CHUNK_SIZE, SPEAKING_SPEED, PASS_PERCENTAGE
        global job_role, candidate_skill, role_description 

        # Get values from input fields
        FS = int(fs_entry.get())
        CHUNK_SIZE = int(chunk_size_entry.get())
        SPEAKING_SPEED = float(speaking_speed_entry.get())
        PASS_PERCENTAGE = float(pass_percentage_entry.get())
        job_role = job_role_entry.get()
        candidate_skill = candidate_skill_entry.get()
        role_description = role_description_text.get("1.0", tk.END).strip()

        # Basic input validation
        if not all([job_role, candidate_skill, role_description]):
            messagebox.showerror("Error", "Please fill in all job details.")
            return

        root.destroy()  # Close the first UI window
        settings_updated.set()  # Signal that settings have been updated

    root = tk.Tk()
    root.title("Job Details and Settings")
    root.geometry("900x400") 
    root.configure(bg='#f0f0f0')

    # Job Details Tile
    job_frame = tk.LabelFrame(root, text="Job Details", bg='#e0e0e0', padx=10, pady=10)
    job_frame.grid(row=0, column=0, padx=10, pady=10, sticky="nsew")

    tk.Label(job_frame, text="Job Role:", bg='#e0e0e0').grid(row=0, column=0, sticky='e', pady=5)
    job_role_entry = tk.Entry(job_frame, width=30)
    job_role_entry.grid(row=0, column=1, sticky='w', pady=5)

    tk.Label(job_frame, text="Candidate Skill:", bg='#e0e0e0').grid(row=1, column=0, sticky='e', pady=5)
    candidate_skill_entry = tk.Entry(job_frame, width=30)
    candidate_skill_entry.grid(row=1, column=1, sticky='w', pady=5)

    tk.Label(job_frame, text="Role Description:", bg='#e0e0e0').grid(row=2, column=0, sticky='ne', pady=5)
    role_description_text = tk.Text(job_frame, width=60, height=10)
    role_description_text.grid(row=2, column=1, sticky='w', pady=5)

    # Settings Tile
    settings_frame = tk.LabelFrame(root, text="Interview Settings", bg='#e0e0e0', padx=10, pady=10)
    settings_frame.grid(row=0, column=1, padx=10, pady=10, sticky="nsew")

    settings = [
        ("Pass %", PASS_PERCENTAGE),
        ("Sampling Frequency", FS),
        ("Chunk Size", CHUNK_SIZE),
        ("Speaking Speed", SPEAKING_SPEED)
    ]

    entries = []
    for i, (text, value) in enumerate(settings):
        tk.Label(settings_frame, text=text, bg='#e0e0e0').grid(row=i, column=0, sticky='e', pady=5)
        entry = tk.Entry(settings_frame, width=10)
        entry.insert(0, str(value))
        entry.grid(row=i, column=1, sticky='w', pady=5)
        entries.append(entry)

    pass_percentage_entry, fs_entry, chunk_size_entry, speaking_speed_entry = entries

    update_button = tk.Button(settings_frame, text="Update Settings", command=update_settings, 
                              bg="blue", fg="white", font=("Arial", 10))
    update_button.grid(row=len(settings), column=0, columnspan=2, pady=10)

    # Configure grid weights
    root.grid_columnconfigure(0, weight=1)
    root.grid_columnconfigure(1, weight=1)
    root.grid_rowconfigure(0, weight=1) 

    root.protocol("WM_DELETE_WINDOW", lambda: settings_updated.set())  # Handle window close
    root.mainloop()

# Start the UI thread
thread1 = Thread(target=create_ui_1)
thread1.start()

# Wait for settings to be updated
settings_updated.wait()

# Continue with the rest of your code here
print("Settings updated, continuing with execution...")
print(f"Job Role: {job_role}")
print(f"Candidate Skill: {candidate_skill}")
print(f"Role Description: {role_description}")
print(f"FS: {FS}")
print(f"CHUNK_SIZE: {CHUNK_SIZE}")
print(f"SPEAKING_SPEED: {SPEAKING_SPEED}")
print(f"PASS_PERCENTAGE: {PASS_PERCENTAGE}")

# Add the rest of your program logic here

Settings updated, continuing with execution...
Job Role: None
Candidate Skill: None
Role Description: None
FS: 44100
CHUNK_SIZE: 1024
SPEAKING_SPEED: 1.0
PASS_PERCENTAGE: 0.7


In [ ]:
# -- Init Directory --
if not os.path.exists("interviews/" + unixtime):
    os.makedirs("interviews/" + unixtime)
    os.makedirs("interviews/" + unixtime + "/audio")
    os.makedirs("interviews/" + unixtime + "/pdfs")
    os.makedirs("interviews/" + unixtime + "/joblib")
audio_directory = "interviews/" + unixtime + "/audio/"
pdf_directory = "interviews/" + unixtime + "/pdfs/"
joblib_directory = "interviews/" + unixtime + "/joblib/"

job_role = "RAG AI Engineer"
candidate_skill = "Entry-Level"
role_description = """
Permanent

London (Hybrid)

Salary - £50,000 - £75,000 p/a + benefits

My client are on the cutting edge of digital reinvention, helping clients reimagine how they serve their connected customers and operate enterprises. As an experienced AI Engineer, you'll play a pivotal role in their revolution. You'll leverage deep learning, neuro-linguistic programming (NLP), computer vision, chatbots, and robotics to enhance business outcomes and drive innovation. Join their multidisciplinary team to shape their AI strategy and showcase the potential of AI through early-stage solutions.

Tasks

1. Enhance Retrieval and Generation:
Create and manage RAG pipelines to improve information retrieval and content generation tasks.
1. LLMs Optimization:
Understand the nuances between prompting and training large language models (LLMs) to enhance model performance.
1. LLM Evaluation:
Evaluate different LLMs to find the best fit for specific use cases.
1. Model Efficiency:
Address speed, performance, and cost-related issues in model implementation.
1. Collaboration and Innovation:
Work closely with cross-functional teams to integrate AI solutions into production environments.
Stay informed about the latest advancements in AI and machine learning to continuously enhance our solutions.
Requirements

4+ years of hands-on Python development experience, especially with machine learning frameworks (e.g., TensorFlow, PyTorch).
Proven experience setting up and optimizing retrieval-augmented generation (RAG) pipelines.
Strong understanding of large language models (LLMs) and the differences between prompting and training.
Production-level experience with AWS services.
Hands-on experience testing and comparing different LLMs (OpenAI, Llama, Claude, etc.).
Familiarity with model speed and cost optimization challenges.
Excellent problem-solving skills and attention to detail.
Strong communication and teamwork abilities.
Benefits

Endless Learning and Growth: Explore boundless opportunities for personal and professional development in our dynamic, AI-driven startup.
Inclusive and Supportive Environment: Join a collaborative culture that prioritizes transparency, trust, and open dialogue among team members.
Generous Benefits: Enjoy comprehensive perks, including unlimited annual leave, birthday leave, and exciting team trips.
Impactful Work: Contribute to the financial industry by working with cutting-edge AI technologies that make a difference.
Please apply for this exciting role ASAP!!
"""

In [ ]:
system_prompt = f"""
You are a skilled interviewer who is conducting an initial phone screening interview for a candidate for a {candidate_skill} {job_role} role to see if the candidate is at minimum somewhat qualified for the role and worth the time to be fully interviewed. The role and company description is copypasted from the job posting as follows: {role_description}. Parse through it to extract any information you feel is relevant.
Your job is to begin a friendly discussion with the candidate, and ask questions relevant to the {job_role} role, which may or may not be based on the interviewee's CV, which you have access to. Be sure to stick to this topic even if the candidate tries to steer the conversation elsewhere. If the candidate has other experience on his CV, you can ask about it, but keep it within the context of the {job_role} role.
After the candidate responds to each of your questions, you should not summarise or provide feedback on their responses. THIS POINT IS KEY! You should not summarise or provide feedback on their responses. You must keep your responses short and concise without reiterating what is good about the candidate's response or experience when they reply.
You can ask follow-up questions if you wish.
Once you have asked sufficient questions such that you deem the candidate is or isn't fitting for the role, end the interview by thanking the candidate for their time and informing them that they will receive word soon on the outcome of the screening interview. If the candidate does not seem fititng for the role, or if something feels off such as the candidate being unconfident or very very vague feel free to end the interview early. There is no need to inform them of your opinion of their performance, as this will be evaluated later.
The candidate will begin the interview by greeting you. You are to greet them back, and begin the interview.
For this specific run, keep the interview to a maximum of 4 questions.
"""

In [ ]:
# Initialising Conversation Model
chat_model_name = "claude-3-5-sonnet-20240620"
cv_path = "docs/cvs/cv-niranj.pdf"

chat_model = ClaudeChatCV(chat_model_name, system_prompt, cv_path)

In [ ]:
# Initialising whisper
stt_model = whisper.load_model("medium", device="cuda")

In [ ]:
# Initialising text2speech
tts_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def stream_tts(input_string):
    def _stream_tts():
        p = pyaudio.PyAudio()
        stream = p.open(format=8,
                        channels=1,
                        rate=round(24_005 * SPEAKING_SPEED),
                        output=True)
        with tts_client.audio.speech.with_streaming_response.create(
            model="tts-1",
            voice="nova",
            input=input_string,
            response_format="pcm"
        ) as response:
            for chunk in response.iter_bytes(1024):
                stream.write(chunk)
                
        thread_done.set()

    thread_done = threading.Event()

    thread = threading.Thread(target=_stream_tts)
    thread.start()
    thread_done.wait()

In [ ]:
# Initialising speech2text without silence detection
def record_speech():
    global human_audio_n
    print("Recording... Speak now!")
    audio_data = np.array([], dtype=np.int16)  # Initialize empty array

    with sd.InputStream(samplerate=FS, channels=1, dtype='int16') as stream:
        while True:
            chunk, overflowed = stream.read(CHUNK_SIZE)
            if overflowed:
                print("Warning: Input overflowed!")
            audio_data = np.append(audio_data, chunk)

            if pause_loop:
                break
    
    human_audio_n += 1
    wavstring = f"/audio_{human_audio_n}_{unixtime}.wav"
    wav.write(audio_directory + wavstring, FS, audio_data)

    return wavstring


In [ ]:
def create_ui():
        def toggle_pause():
            global pause_loop
            pause_loop = not pause_loop
            pause_button.config(text="Stop" if not pause_loop else "Speak now",
                                bg="red" if not pause_loop else "green")

        def end_program():
            global end_loop
            end_loop = True
            root.quit()
            root.destroy()

        root = tk.Tk()
        root.title("Control Panel")
        root.geometry("300x200")
        root.configure(bg='#f0f0f0')

        frame = tk.Frame(root, bg='#f0f0f0')
        frame.pack(expand=True, fill='both', padx=20, pady=20)

        pause_button = tk.Button(frame, text="Stop", command=toggle_pause, 
                                bg="red", fg="white", font=("Arial", 12), 
                                width=10, height=2)
        pause_button.pack(pady=10)

        end_button = tk.Button(frame, text="End Program", command=end_program, 
                            bg="gray", fg="white", font=("Arial", 12), 
                            width=10, height=2)
        end_button.pack(pady=10)

        root.protocol("WM_DELETE_WINDOW", end_program)
        root.mainloop()
        
ui_thread = Thread(target=create_ui)
ui_thread.start()

In [ ]:
while True:
    if not pause_loop:
        # --- Record Speech ---
        time.sleep(0.1)
        print("Recording speech...")
        wav_file = record_speech()

        if end_loop:
            break

        # --- Speech to Text ---
        print("Converting speech to text...")
        text = stt_model.transcribe(audio_directory + wav_file, language="en")
        print("You said: ", text.get("text"))

        if end_loop:
            break

        # --- Chatbot ---
        print("Chatting...")
        response = chat_model.chat_with_history_doc(text.get("text"))

        print("Chatbot: ", response)

        # --- Text to Speech ---
        print("Converting text to speech...")
        stream_tts(response)

    else:
        time.sleep(0.1)
        if end_loop:
            break

Recording speech...
Recording... Speak now!


KeyboardInterrupt: 

In [ ]:
conversation = chat_model.get_message_history()
dump(conversation, joblib_directory + "conversation.joblib")


print(conversation)

In [ ]:
import re
from fpdf import FPDF

# Create a PDF object
pdf = FPDF()
pdf.add_page()

# Set margins (in millimeters)
pdf.set_margins(left=10, top=20, right=10)  # Right margin set to 10mm 

# Choose a nicer font
pdf.set_font("Helvetica", size=12)

# Calculate usable width for text wrapping (accounting for margins)
usable_width = pdf.w - pdf.l_margin - pdf.r_margin -0 # 10px right margin 

# Add conversation to the PDF
for turn in conversation[2:]:
    # Skip turns with empty roles or content
    if turn['role'] and turn['content']:
        # Subheading style for "User" and "Assistant"
        pdf.set_font("Helvetica", style="B", size=14)
        pdf.cell(0, 10, txt=f"{turn['role'].capitalize()}:", ln=True)

        # Content with regular font and correct indentation
        pdf.set_font("Helvetica", size=12)
        pdf.x = pdf.l_margin  # Reset x-coordinate to the left margin
        pdf.multi_cell(usable_width, 6, txt=turn['content'])
        pdf.ln(3)

# Save the PDF
pdf.output(pdf_directory + "conversation.pdf")